In [ ]:
import pandas as pd
import os 
import matplotlib.pyplot as plt
import numpy as np
from typing import Optional, Union
from event_detectors.precipitation_detector import precip_event_detector as pcp
from event_detectors.soil_moisture_detector import soil_moisture_event_detector as sdd

In [ ]:
RESULTS = ".\\..\\..\\results_v2"
SENSOR_DEPTH = pd.read_csv("../../../data/NEON_soil_moisture/swc_depthsV2.csv") # NEON sensor depth dataset
# processing the soil moisture data.
SM_DATA_PARAMS = {
                "sample_period":"10min",
                "min_periods":7,
                "interpolate":True,
                "max_period":36
                }
# processing precipitation dataset. 
PRECIP_PARAMS = {
                    "sample_period": "10min",
                    "max_nan_allowed": 5,
                    "min_termination": 6,
                    "sum_amount":2,
                    "max_end": 5,
                    }

In [ ]:
def table_creater(save_file = True):
    sites = []
    path = "../../../extracted_data/sm/"
    files = os.listdir(path)
    for i in files:
        name = i.split("_sm_")[0]
        data = pd.read_csv(path + i)
        data["startDateTime"] = pd.to_datetime(data["startDateTime"])
        time_diff = (data.iloc[-1,0] -  data.iloc[0,0]).total_seconds()/ (365.25*24*60*60)
        sites.append([name, time_diff])
    final_data = pd.DataFrame(sites, columns=["site", "years"])
    final_data = final_data.groupby("site", as_index=False).agg({"years":"mean"})
    if save_file:
        final_data.to_csv("data_length.csv", index=False)
    return final_data
# table_creater(save_file = False)

In [ ]:
def process_sm_data(site_name:str, params:dict) -> pd.DataFrame:
    """process soil moisture i.e., interpolation, data curation.

    Args:
        site_name (str): site name.
        params (dict): parameters associated with soil moisture data processing.

    Returns:
        pd.DataFrame: processed soil moisture data.
    """
    raw_sm = sdd.sm_loader(site_name)
    flagged_sm = sdd.sm_flagger(raw_sm)
    resampled_sm = sdd.sm_resampler(
                                flagged_sm,
                                sample_period = params.get("sample_period"), # 10min
                                min_periods = params.get("min_periods"),  # 7
                                interploate = params.get("interpolate"), # True
                                max_period =  params.get("max_period")  #  36
                                ) #fill gaps <= 6 hours with linear interpolation 
                                
    return resampled_sm

In [ ]:
def process_precip_event(site_name:str, params:dict) -> pd.DataFrame:
    """process of precipitation dataset.

    Args:
        site_name (str): site name.
        params (dict): dictionary of precip parameters.

    Returns:
        pd.DataFrame: precipitation events.
    """
    raw_precip = pcp.precip_loader(site_name)
    flagged_precip = pcp.precip_flagger(raw_precip)
    resampled_precip = pcp.precip_resampler(
                                            flagged_precip,
                                            sample_period = params.get("sample_period"), # 10min
                                            max_nan_allowed = params.get("max_nan_allowed") # 5
                                            )
    return resampled_precip

In [ ]:
# def get_all_ts(run_save = False):
#     """save soil moisture and precipitation time series or not. 

#     Args:
#         run_save (bool, optional): if these files are saved or not. Defaults to False.
#     """
#     NEON_SITES = sorted(list(set([i.split("_")[0] for i in os.listdir("../../../extracted_data/sm")])))
#     if run_save:
#         for i in NEON_SITES:
#             sm = process_sm_data(i, SM_DATA_PARAMS)
#             precip = process_precip_event(i, PRECIP_PARAMS)
#             for j in sm:
#                 sm[j].to_csv(f"E:\AI4PF\AI4PF_man\data\clean_sm\{j}.csv", index=True)
#             precip.to_csv(f"E:\AI4PF\AI4PF_man\data\clean_precip\{i}_clean_precip.csv", index=True)
# get_all_ts()

In [ ]:
def get_sensor_depth(site:str, hposition:str, vposition:str, 
                     time:Union[pd.Series, pd.Timestamp, None]) -> Union[float, int, pd.DataFrame]:
    """get sensor depth information.

    Args:
        site (str): NEON site.
        hposition (str): soil plot number. e.g., "001" - "005
        vposition (str): sensor position. e.g., "501" etc

    Returns:
        Union[float, int, pd.DataFrame]: sensor depth. Use none type while the sensor is not moving around.
    """
    
    sensor_depth = SENSOR_DEPTH.copy()
    sensor_depth = sensor_depth.rename(columns={"horizontalPosition.HOR":"hPosition",
                                                "verticalPosition.VER":"vPosition"})
    
    sensor_depth["startDateTime"] = pd.to_datetime(sensor_depth["startDateTime"], format="%Y-%m-%dT%H:%M:%SZ")
    sensor_depth["endDateTime"] = pd.to_datetime(sensor_depth["endDateTime"], format="%Y-%m-%dT%H:%M:%SZ")
    # convert the hoizontal position and vertical position field to strings 
    # sensor depth was stored in m, here converts to cm. 
    sensor_depth["sensorDepth"] = np.abs(sensor_depth["sensorDepth"]) * 100
    sensor_depth["hPosition"] = sensor_depth["hPosition"].apply(lambda x: f"00{x}")
    sensor_depth["vPosition"] = sensor_depth["vPosition"].apply(str) 
    
    candidate_depth = sensor_depth[(sensor_depth["siteID"] == site) &
                                   (sensor_depth["hPosition"] == hposition) &
                                   (sensor_depth["vPosition"] == vposition)].copy()
    # don't apply series at all 
    
    def nest_loop(r, data):
        for _, row in data.iterrows():
            if row["startDateTime"] <= r < row["endDateTime"]:
                return row["sensorDepth"]
                
            if row["startDateTime"] <= r and pd.isna(row["endDateTime"]):
                return row["sensorDepth"]
            
        return np.nan
    
    # if specify None meaning the values can be nan
    if time is None:
        assert candidate_depth.shape[0] == 1, "the sensor has multiple depth."
        return candidate_depth["sensorDepth"].values[0]
    elif isinstance(time, pd.Timestamp):
        return nest_loop(time, candidate_depth)
    elif isinstance(time, pd.Series):
        return time.apply(nest_loop, args=(candidate_depth, ))
    else:
        raise TypeError("input type is not accepted")

In [ ]:
SITE_EXAMPLE = "ABBY"
PRECIP_DATA = process_precip_event(SITE_EXAMPLE, PRECIP_PARAMS)
SM_DATA =  process_sm_data(SITE_EXAMPLE, SM_DATA_PARAMS)

In [ ]:
def plot_ts(site_name, plot_id, start_time, end_time):
    res = pd.read_csv(os.path.join(RESULTS, f"{site_name}_result_{plot_id}.csv"))
    col_with_t = [i for i in res.columns if "Time" in i]
    res.loc[:,col_with_t] = res.loc[:,col_with_t].apply(pd.to_datetime)
    fig, ax = plt.subplots(4,figsize = (10, 8))
    precip_spliter = res.loc[(res["stormStartTime"] >= start_time) & (res["stormEndTime"] <= end_time), 
                        ["stormStartTime", "stormEndTime"]]
    
    precip_data = PRECIP_DATA.loc[start_time:end_time]
    sm_data = SM_DATA[f"{site_name}_sm_{plot_id}"]
    ax[0].bar(precip_data.index, precip_data * 6,.25, color = "grey")
    ax[0].set_ylabel("Rainfall (mm/h)")
    fg_num = ["a", "b", "c", "d"]
    ax[0].text(0.05, 0.8, fg_num[0], transform = ax[0].transAxes,fontsize = 15)

    for index, row in precip_spliter.iterrows():
        ax[0].axvline(x=row["stormStartTime"], color='red', linestyle='--', linewidth=1, label = "Rainfaill starts")
        ax[0].axvline(x=row["stormEndTime"], color='green', linestyle='--', linewidth=1, label = "Rainfaill ends")
    ax[0].legend(["Rainfall starts", "Rainfall ends"], loc = "upper right")
    for i in [1,2,3]:
        data = sm_data.loc[start_time:end_time, f"VSWCMean_50{i}"]
        depth = get_sensor_depth(site_name, hposition=plot_id, vposition=f"50{i}", time = start_time)
        depth = int(depth)
        res_onset =  res.loc[(res["stormStartTime"] >= start_time) & (res["stormEndTime"] <= end_time), 
                         [f"smOnsetTime_50{i}", f"smAtOnset_50{i}"
                          ]].dropna()
        res_peak =  res.loc[(res["stormStartTime"] >= start_time) & (res["stormEndTime"] <= end_time), 
                         [ f"smPeakTime_50{i}", f"smAtPeak_50{i}"
                          ]].dropna()
        ax[i].plot(data.index, data)
        ax[i].scatter(res_onset[f"smOnsetTime_50{i}"], res_onset[f"smAtOnset_50{i}"], c= "blue", label = "Onsets")
        ax[i].scatter(res_peak[f"smPeakTime_50{i}"], res_peak[f"smAtPeak_50{i}"], c= "red", label = "Peaks")
        ax[i].legend(loc= "upper right")
        ax[i].set_ylabel(f"Soil moisture (v/v)")
        ax[i].text(0.2, 0.85, f"sensor depth = {depth} cm", transform = ax[i].transAxes)
        ax[i].text(0.05, 0.8, fg_num[i], transform = ax[i].transAxes, fontsize = 15)
#plot_ts(SITE_EXAMPLE, "001", pd.to_datetime('2017-05-15 00:00:00'), pd.to_datetime("2017-07-30 00:00:00"))

In [ ]:
def create_multiple_site_data(site):
   
    precip = process_precip_event(site, PRECIP_PARAMS)
    sm =  process_sm_data(site, SM_DATA_PARAMS)
    
    return precip, sm[f"{site}_sm_001"]

In [ ]:
def get_multi_site_pf(site):
    path = "E:/AI4PF/doc/PF_database_v2/"
    respone = ["noResponse", "sensorResponded"]
    
    res = pd.read_csv(os.path.join(path, f"{site}_PF_database_001.csv"))
    for col in res.columns:   
        if ("smOnsetTime") in col or ("stormStartTime"in col) or ("stormEndTime" in col):
            res[col] = pd.to_datetime(res[col], format = " %d.%m.%Y %H:%M:%S")
    
    sensor_num = [i for i in res.columns if "smOnsetTime" in i]
    # needed precipitation information
    precipi_info = res[['stormStartTime', 'stormEndTime', 'stormStartValue', 'stormEndValue']].copy()
    
    # sensor wise
    sensor_information = {}
    sensor_information["pf_info"] = precipi_info
    for j in np.arange(1, len(sensor_num) + 1):
        sensor_data = res.loc[res[f"smResponseType_50{j}"].isin(respone),
                                ['stormStartTime', 'stormEndTime',
                                f"smResponseType_50{j}", f"PF_velocity_metric_50{j}"]]
        sensor_information[f"50{j}"] = sensor_data
    
    return sensor_information

In [ ]:
def plot(precipitation, soil_moisture, site):
    # pcp, sm = create_multiple_site_data(site)
    precipitation = precipitation.copy().to_frame()
    precipitation = precipitation.reset_index()
    precipitation["color"] = "red"
    events = get_multi_site_pf(site)
    print(site)
    print(events["pf_info"][events["pf_info"]["stormStartTime"] > events["pf_info"]["stormEndTime"]])
    for j in soil_moisture.columns:
        name = j.split("_")[-1]
        sm_j = soil_moisture[j].to_frame().copy()
        pf_j = events[name]
        pf_j = pf_j[pf_j["stormStartTime"] <= pf_j["stormEndTime"]]
        # Initialize the color and status columns
        sm_j['color'] = 'grey'  # Default color
        sm_j['status'] = 'No event'  # Default status
        fig2 = go.Figure()
        # Loop through each storm event
        fig2.add_trace(go.Scatter(
                x=sm_j.index,
                y=sm_j[j],
                mode='lines', 
                line={'color':'grey'},
                xaxis= "x",
                yaxis= "y",
                name = "Non event"
            ))

        for _, e in pf_j.iterrows():
            mask = (sm_j.index >= e['stormStartTime']) & (sm_j.index <= e['stormEndTime'])
            color = "green"
            event_type = "non PF event"
            
            if e[f"PF_velocity_metric_{name}"]:
                color = 'fuchsia'
                event_type = 'PF event'
                
           
            fig2.add_trace(go.Scatter(
                x=sm_j[mask].index,
                y=sm_j[mask][j],
                mode='lines', 
                line={'color':color},
                showlegend=False,
                xaxis= "x",
                yaxis= "y",
            ))
           
        
        # Add bar trace for precipitation
        fig2.add_trace(go.Bar(
            x=events["pf_info"]["stormStartTime"],
            y=events["pf_info"]["stormStartValue"],
            width=20,
            name='Precipitation start',  # Add a name for the legend
            marker_line_color = "white",
           
            yaxis='y2'  # Link this trace to the first y-axis
        ))
        
        fig2.add_trace(go.Bar(
            x=events["pf_info"]["stormEndTime"].tolist(),
            y=events["pf_info"]["stormEndValue"].tolist(),
            width=20,
            name='Precipitation end',  # Add a name for the legend
            marker_line_color = "orange", 
         
            yaxis='y2'  # Link this trace to the first y-axis
        ))

        fig2.update_layout(
        # Configuration for the primary y-axis
            yaxis=dict(
                title='Soil moisture [-]',
                showgrid=False,
                titlefont=dict(size=18, color='white'), 
            ),
        # Configuration for the secondary y-axis
            yaxis2=dict(
                title='Precipitation event',
                overlaying='y',  # Overlay on the same plot area
                side='right', # Place on the right side
                autorange = "reversed",
                showgrid=False
            ),
            xaxis=dict(
                rangeslider=dict(
                    visible=True  # Show the range slider
                ),
                type='date'  # Ensure the x-axis is treated as a date axis
            ),
            plot_bgcolor='black',
        )
        fig2.write_html(f'E:/AI4PF/doc/scripts/python/{site}_{j}.html')
    
    
for s in ["ABBY", "ORNL", "ONAQ", "SOAP"]:
    df_pre, df_sm = create_multiple_site_data(s)
    plot(df_pre, df_sm, s)